# 🌍 Sistema de Geolocalização de Imóveis

**Objetivo:** Encontrar o endereço exato de um imóvel a partir de uma foto.

## 🎯 O que este notebook faz:

1. Analisa a foto do imóvel (arquitetura, cores, contexto)
2. Busca candidatos na área (Google Places + Grid Search)
3. Baixa imagens do Street View
4. Compara visualmente (CLIP + SIFT)
5. Valida com LLM (Claude)
6. Retorna endereço completo com confiança

---

## 📦 Instalação

In [ ]:
# Instalar dependências (rode apenas uma vez)
!pip install -q anthropic requests pillow numpy opencv-python-headless
!pip install -q torch open-clip-torch pandas folium tqdm python-dotenv

## 🔑 Configuração de APIs

In [ ]:
import os

# Configure suas chaves de API aqui
os.environ["GOOGLE_API_KEY"] = "sua_chave_google_aqui"
os.environ["ANTHROPIC_API_KEY"] = "sua_chave_anthropic_aqui"

print("✅ APIs configuradas!")

## 📸 Upload da Foto do Imóvel

In [ ]:
from google.colab import files
from PIL import Image
import io

# Upload da foto
uploaded = files.upload()

# Pegar nome do arquivo
foto_path = list(uploaded.keys())[0]

# Visualizar
img = Image.open(foto_path)
img.thumbnail((600, 600))
display(img)

print(f"\n📸 Foto carregada: {foto_path}")

## 🚀 Inicializar Sistema

In [ ]:
import sys
from pathlib import Path

# Adicionar diretório do projeto ao path
sys.path.insert(0, '/content/geolocaliza')

from main import GeoLocalizador

# Inicializar
geo = GeoLocalizador()

print("✅ Sistema inicializado!")

## 🎯 Executar Geolocalização

In [ ]:
# Configurar área de busca
cidade = "São Paulo"
bairro = "Alto da Boa Vista"  # Opcional
center_lat = -23.6505  # Coordenada central
center_lon = -46.6815
radius_m = 2000  # Raio de busca em metros

print("🔍 Iniciando busca...\n")
print(f"📍 Cidade: {cidade}")
print(f"🏘️  Bairro: {bairro}")
print(f"📏 Raio: {radius_m}m\n")
print("⏱️  Isso pode levar 2-5 minutos...\n")

# Executar
resultado = geo.localizar_imovel(
    foto_path=foto_path,
    cidade=cidade,
    bairro=bairro,
    center_lat=center_lat,
    center_lon=center_lon,
    radius_m=radius_m
)

## 📊 Resultado

In [ ]:
import json

if resultado["success"]:
    print("\n" + "="*60)
    print("✅ IMÓVEL LOCALIZADO!")
    print("="*60 + "\n")
    
    print(f"📍 Endereço Completo:")
    print(f"   {resultado['endereco']}\n")
    
    print(f"🏠 Detalhes:")
    print(f"   Rua: {resultado['rua']}")
    print(f"   Número: {resultado['numero']}")
    print(f"   Bairro: {resultado['bairro']}")
    print(f"   Cidade: {resultado['cidade']}")
    print(f"   Estado: {resultado['estado']}")
    print(f"   CEP: {resultado['cep']}\n")
    
    print(f"🎯 Confiança: {resultado['confianca']:.1%}\n")
    
    print(f"📊 Scores Detalhados:")
    print(f"   CLIP (semântica): {resultado['scores']['clip']:.3f}")
    print(f"   Geometria (SIFT): {resultado['scores']['geometria']:.3f}")
    print(f"   LLM (validação): {resultado['scores']['llm']:.3f}\n")
    
    print(f"📌 Coordenadas:")
    print(f"   Lat: {resultado['coordenadas']['lat']:.6f}")
    print(f"   Lon: {resultado['coordenadas']['lon']:.6f}\n")
    
    print(f"💭 Raciocínio do Sistema:")
    print(f"   {resultado['reasoning']}\n")
    
    print(f"🗺️  Street View:")
    print(f"   {resultado['street_view_link']}\n")
    
    print("="*60)
    
else:
    print("\n" + "="*60)
    print("❌ NÃO FOI POSSÍVEL LOCALIZAR")
    print("="*60 + "\n")
    print(f"⚠️  Erro: {resultado['error']}\n")
    if 'hint' in resultado:
        print(f"💡 Dica: {resultado['hint']}\n")
    print("="*60)

## 🗺️ Visualizar Mapa Interativo

In [ ]:
from IPython.display import IFrame

# Exibir mapa gerado
mapa_path = "/content/geolocaliza/output/mapa.html"

if Path(mapa_path).exists():
    print("🗺️  Mapa interativo:")
    display(IFrame(mapa_path, width=800, height=600))
else:
    print("⚠️  Mapa não foi gerado")

## 📁 Análise Visual Detalhada

In [ ]:
import json

# Carregar análise visual
analise_path = "/content/geolocaliza/output/analise_visual.json"

if Path(analise_path).exists():
    with open(analise_path, 'r', encoding='utf-8') as f:
        analise = json.load(f)
    
    print("🔍 ANÁLISE VISUAL DA FOTO\n")
    print("="*60)
    
    # Arquitetura
    arq = analise['analysis']['architecture']
    print("\n🏛️  ARQUITETURA:")
    print(f"   Estilo: {arq['style']}")
    print(f"   Andares visíveis: {arq['floors_visible']}")
    print(f"   Tipo de telhado: {arq['roof_type']}")
    print(f"   Cor principal: {arq['main_color']}")
    print(f"   Material: {arq['material']}")
    
    # Elementos distintivos
    dist = analise['analysis']['distinctive_features']
    print("\n✨ ELEMENTOS DISTINTIVOS:")
    print(f"   Tipo de portão: {dist['gate_type']}")
    print(f"   Janelas: {dist['windows']['style']} ({dist['windows']['count_visible']} visíveis)")
    print(f"   Varanda/Garagem: {dist['balcony_garage']}")
    print(f"   Jardim/Plantas: {dist['garden_plants']}")
    if dist['unique_elements']:
        print(f"   Únicos: {', '.join(dist['unique_elements'])}")
    
    # Contexto urbano
    ctx = analise['analysis']['urban_context']
    print("\n🏙️  CONTEXTO URBANO:")
    print(f"   Tipo de rua: {ctx['street_type']}")
    print(f"   Calçada: {ctx['sidewalk']}")
    print(f"   Árvores: {ctx['trees_visible']}")
    print(f"   Postes: {ctx['utility_poles']}")
    print(f"   Prédios adjacentes: {ctx['adjacent_buildings']}")
    print(f"   Inclinação: {ctx['street_slope']}")
    
    # Textos visíveis
    txt = analise['analysis']['visible_text']
    print("\n📝 TEXTOS VISÍVEIS:")
    print(f"   Número: {txt['address_number']}")
    if txt['street_signs']:
        print(f"   Placas: {', '.join(txt['street_signs'])}")
    print(f"   Nome do condomínio: {txt['condo_name']}")
    
    print("\n" + "="*60)
else:
    print("⚠️  Análise visual não disponível")

## 📊 Análise dos Candidatos

In [ ]:
import pandas as pd

# Carregar candidatos validados
candidatos_path = "/content/geolocaliza/output/candidatos_validados.csv"

if Path(candidatos_path).exists():
    df = pd.read_csv(candidatos_path)
    
    print("\n📊 TOP 5 CANDIDATOS:\n")
    print("="*80)
    
    for i, row in df.head(5).iterrows():
        print(f"\n#{i+1} - Confiança: {row['final_confidence']:.1%}")
        print(f"   Match LLM: {'✅ SIM' if row['llm_is_match'] else '❌ NÃO'}")
        print(f"   Scores: CLIP={row['clip_score']:.3f} | GEOM={row['geom_score']:.3f} | LLM={row['llm_confidence']:.3f}")
        print(f"   Coordenadas: {row['lat']:.6f}, {row['lon']:.6f}")
        print(f"   Raciocínio: {row['llm_reasoning']}")
        if pd.notna(row.get('name')):
            print(f"   Local: {row['name']}")
    
    print("\n" + "="*80)
    
    # Estatísticas
    print("\n📈 ESTATÍSTICAS:\n")
    print(f"   Total de candidatos: {len(df)}")
    print(f"   Matches LLM: {df['llm_is_match'].sum()}")
    print(f"   Score CLIP médio: {df['clip_score'].mean():.3f}")
    print(f"   Score Geométrico médio: {df['geom_score'].mean():.3f}")
    print(f"   Confiança final média: {df['final_confidence'].mean():.3f}")
    
else:
    print("⚠️  Dados de candidatos não disponíveis")

## 🖼️ Visualizar Match Visual

In [ ]:
from PIL import Image
import matplotlib.pyplot as plt

if resultado["success"]:
    # Carregar foto original
    img_query = Image.open(foto_path)
    
    # Pegar melhor match
    df = pd.read_csv("/content/geolocaliza/output/candidatos_validados.csv")
    best = df.iloc[0]
    
    # Carregar Street View correspondente
    sv_path = f"/content/geolocaliza/output/street_views/{best['filename']}"
    
    if Path(sv_path).exists():
        img_sv = Image.open(sv_path)
        
        # Plotar lado a lado
        fig, axes = plt.subplots(1, 2, figsize=(16, 8))
        
        axes[0].imshow(img_query)
        axes[0].set_title(f"FOTO DO USUÁRIO\n{foto_path}", fontsize=14, fontweight='bold')
        axes[0].axis('off')
        
        axes[1].imshow(img_sv)
        axes[1].set_title(
            f"MELHOR MATCH (Street View)\n"
            f"Confiança: {best['final_confidence']:.1%} | "
            f"CLIP: {best['clip_score']:.3f} | "
            f"GEOM: {best['geom_score']:.3f}",
            fontsize=14,
            fontweight='bold'
        )
        axes[1].axis('off')
        
        plt.tight_layout()
        plt.show()
    else:
        print("⚠️  Imagem do Street View não encontrada")
else:
    print("⚠️  Nenhum match para visualizar")

## 💾 Baixar Resultados

In [ ]:
from google.colab import files
import shutil

# Criar ZIP com todos os resultados
shutil.make_archive('/content/resultados_geolocalizacao', 'zip', '/content/geolocaliza/output')

print("📦 Baixando resultados...")
files.download('/content/resultados_geolocalizacao.zip')

print("\n✅ Download completo!")
print("\n📁 O ZIP contém:")
print("   • resultado_final.json")
print("   • analise_visual.json")
print("   • candidatos.csv")
print("   • candidatos_validados.csv")
print("   • mapa.html")
print("   • street_views/ (imagens)")
print("   • geolocaliza.log")

## 🔧 Teste Individual de Agentes

Para debugar problemas, teste cada agente separadamente:

In [ ]:
# Teste do VisionAgent
from agents.vision_agent import VisionAgent

vision = VisionAgent()
result = vision.analyze_image(foto_path)

if result["success"]:
    print("✅ VisionAgent OK")
    print(json.dumps(result["analysis"], indent=2, ensure_ascii=False))
else:
    print(f"❌ Erro: {result['error']}")

---

## 📚 Documentação Completa

Para mais informações, consulte:
- `README.md` - Visão geral do sistema
- `GUIA_USO.md` - Guia detalhado de uso
- `ARQUITETURA.md` - Arquitetura técnica

---

## 🐛 Problemas Comuns

### "Nenhum candidato encontrado"
→ Aumente o `radius_m` ou verifique coordenadas

### "Confiança baixa"
→ Tire foto de outro ângulo ou reduza `min_confidence` em `config.py`

### "API Error"
→ Verifique as chaves de API e quotas no Google Cloud

---

**Desenvolvido com ❤️ usando Claude + OpenCLIP + Google APIs**